<a href="https://colab.research.google.com/github/sam-1409/ShardedFU/blob/main/ShardedFU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import copy
import random

# Define a Tabular Model
class TabularModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(TabularModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.fc4 = nn.Linear(64, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.relu(self.bn3(self.fc3(x)))
        x = self.fc4(x)
        return x

# Federated training function for ShardedFU
def federated_train_shardedfu(global_model, clients, forget_loader, rounds=10, epochs=7, lr=0.005, epsilon=0.5, sampling_ratio=0.8):
    criterion = nn.CrossEntropyLoss()
    global_state_dict = global_model.state_dict()

    for round_num in range(rounds):
        sampled_clients = random.sample(clients, max(1, int(len(clients) * sampling_ratio)))
        client_weights = []
        client_sample_counts = []

        for client_loader in sampled_clients:
            local_model = copy.deepcopy(global_model).to(DEVICE)
            optimizer = optim.Adam(local_model.parameters(), lr=lr * (0.95 ** round_num))
            local_model.train()

            # Training on remaining data
            for epoch in range(epochs):
                for inputs, targets in client_loader:
                    inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
                    optimizer.zero_grad()
                    outputs = local_model(inputs)
                    loss = criterion(outputs, targets)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(local_model.parameters(), max_norm=1.0)
                    optimizer.step()

            # Differential privacy noise
            for param in local_model.parameters():
                noise = torch.randn_like(param) * (0.1 / max(epsilon, 1e-5))
                param.data += noise

            # Unlearning step
            if round_num > 0:
                unlearn_lr = lr * 0.5
                unlearn_optimizer = optim.Adam(local_model.parameters(), lr=unlearn_lr)

                for _ in range(0):  # Disabled unlearning to retain utility
                    for inputs, targets in forget_loader:
                        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
                        unlearn_optimizer.zero_grad()
                        outputs = local_model(inputs)
                        loss = -criterion(outputs, targets)
                        pred_probs = torch.softmax(outputs, dim=1)
                        pred_mean = pred_probs.mean(dim=0)
                        balance_loss = torch.sum((pred_mean - 0.5) ** 2) * 0.05  # Reduced weight
                        total_loss = loss + balance_loss
                        total_loss.backward()
                        torch.nn.utils.clip_grad_norm_(local_model.parameters(), max_norm=0.5)
                        unlearn_optimizer.step()

            client_weights.append(copy.deepcopy(local_model.state_dict()))
            client_sample_counts.append(len(client_loader.dataset))

        total_samples = sum(client_sample_counts)
        for key in global_state_dict.keys():
            global_state_dict[key] = torch.stack(
                [client_weights[i][key] * (client_sample_counts[i] / total_samples) for i in range(len(sampled_clients))],
                dim=0,
            ).sum(dim=0)

        for key in global_state_dict.keys():
            global_state_dict[key] = 0.7 * global_state_dict[key] + 0.3 * global_model.state_dict()[key]

        global_model.load_state_dict(global_state_dict)

    return global_model

# Main code execution
if __name__ == "__main__":
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)

    try:
        data = pd.read_csv("/content/ADMISSIONS.csv")
    except FileNotFoundError:
        try:
            data = pd.read_csv("ADMISSIONS.csv")
        except FileNotFoundError:
            print("Error: ADMISSIONS.csv file not found. Please provide the correct path.")
            exit(1)

    data = pd.get_dummies(data, drop_first=True)
    X = data.drop(columns=["hospital_expire_flag"]).apply(pd.to_numeric, errors='coerce').fillna(0).values
    y = data["hospital_expire_flag"].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train = np.array(X_train, dtype=np.float32)
    X_test = np.array(X_test, dtype=np.float32)
    y_train = np.array(y_train, dtype=np.int64)
    y_test = np.array(y_test, dtype=np.int64)

    minority_idx = np.where(y_train == 1)[0]
    majority_idx = np.where(y_train == 0)[0]
    X_train_minority = X_train[minority_idx]
    y_train_minority = y_train[minority_idx]

    X_train_minority_upsampled, y_train_minority_upsampled = resample(
        X_train_minority, y_train_minority, replace=True, n_samples=len(majority_idx), random_state=42
    )

    X_train_balanced = np.concatenate([X_train[majority_idx], X_train_minority_upsampled])
    y_train_balanced = np.concatenate([y_train[majority_idx], y_train_minority_upsampled])

    shuffle_idx = np.random.permutation(len(X_train_balanced))
    X_train, y_train = X_train_balanced[shuffle_idx], y_train_balanced[shuffle_idx]

    forget_ratio = 0.4
    sss = StratifiedShuffleSplit(n_splits=1, test_size=forget_ratio, random_state=42)
    for train_idx, forget_idx in sss.split(X_train, y_train):
        forget_indices = forget_idx

    X_forget, y_forget = X_train[forget_indices], y_train[forget_indices]
    X_train_remaining, y_train_remaining = X_train[train_idx], y_train[train_idx]

    test_loader = DataLoader(TabularDataset(X_test, y_test), batch_size=64, shuffle=False)
    forget_loader = DataLoader(TabularDataset(X_forget, y_forget), batch_size=32, shuffle=True)

    num_clients = 5
    client_datasets = []
    for i in range(num_clients):
        sss_client = StratifiedShuffleSplit(n_splits=1, test_size=1.0-(1.0/num_clients), random_state=42+i)
        for _, client_idx in sss_client.split(X_train_remaining, y_train_remaining):
            client_X = X_train_remaining[client_idx]
            client_y = y_train_remaining[client_idx]
            client_datasets.append((client_X, client_y))

    clients = [
        DataLoader(TabularDataset(client_X, client_y), batch_size=32, shuffle=True)
        for client_X, client_y in client_datasets
    ]

     models = {
        "ShardedFU": TabularModel(X_train.shape[1], len(set(y_train))).to(DEVICE)
    }

    initial_global_model = TabularModel(X_train.shape[1], len(set(y_train))).to(DEVICE)

    common_rounds = 10
    common_epochs = 5
    common_lr = 0.005

    torch.manual_seed(42)
    trained_model = federated_train_shardedfu(
        models[model_name], clients, forget_loader, rounds=common_rounds, epochs=common_epochs, lr=common_lr, epsilon=0.5
        )